In [6]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 0
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+fname+'red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])

In [4]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
p_fingerprints[(p_fingerprints==0)] = -1

c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)

#Normalise the features
c_fingerprints = (c_fingerprints - np.mean(c_fingerprints,axis=0))/np.std(c_fingerprints,axis=0)

fingerprints = np.concatenate((p_fingerprints,c_fingerprints),axis=1)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = fingerprints.shape
labels = labels[1:]

In [8]:
labels2 = np.zeros((len(labels),))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)
total_pos = np.sum(labels2)
print("tot_positive",total_pos)


('tot_positive', 48)


In [36]:
clf = RandomForestClassifier(n_estimators = 20,max_depth=2, random_state=0,class_weight={0:1,1:50})
clf.fit(fingerprints,labels2)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 50},
            criterion='gini', max_depth=2, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [37]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints_test = []
c_fp = []
labels = []
with open(path+'AID362red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints_test.append(row[:bf])
        c_fp.append(row[bf:-1])
        labels.append(row[-1])

In [38]:
p_fingerprints_test = np.asarray(p_fingerprints_test)[1:]
p_fingerprints_test = p_fingerprints_test.astype(int)
c_fp = np.asarray(c_fp)[1:]
c_fp = c_fp.astype(float)

c_fp = (c_fp - np.mean(c_fp,axis=0))/np.std(c_fp,axis=0)

fingerprints_test = np.concatenate((p_fingerprints_test,c_fp),axis=1)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints_test.shape
labels = labels[1:]
print(no_examples,ip_dim)
print("total no of 1s",np.sum(p_fingerprints_test))
print("total no of 0s",no_examples*ip_dim-np.sum(p_fingerprints_test))

(856, 112)
('total no of 1s', 6305)
('total no of 0s', 89567)


In [39]:
#p_fingerprints[(p_fingerprints==0)] = -1

labels2_test = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2_test[i] = 1
    else:
        labels2_test[i] = 0
labels2_test = np.asarray(labels2_test,dtype=np.int)

In [40]:
test_op = clf.predict(fingerprints_test)
cm1 = metrics.confusion_matrix(labels2_test,test_op)
print('tn, fp, fn, tp',cm1.ravel())

('tn, fp, fn, tp', array([830,  14,   8,   4]))
